In [4]:
#read the survey dataset
#might take some time as the data is large in size
import pandas as pd
df = pd.read_excel(r"Data.xlsx")
df.head()

,Gender,Age Group,Marital Status,Employment Status,Household Income,Level of Education,Country,Place of Residence Population,Αttacked by a Cyber Attack,Place of the last Incident,Day,Month,Type of Malware,Economic Impacts,Technical Impacts,Type of Device,Type of Operating System,Malware Protection
0,Female,35-44 years old,Other - Don't Know - Prefer Not To Answer - N/A,Unemployed and not currently looking for work,€40.000 - €49.999,Other - Don't Know - Prefer Not To Answer - N/A,China,1.000.001 - 5.000.000 people,Other - Don't Know - Prefer Not To Answer - N/A,Home,Thursday,June,Spyware,Yes,Other - Don't Know - Prefer Not To Answer - N/A,Laptop,Apple iOS,No
1,Male,55-64 years old,Separated,Homemaker,€70.000 - €79.999,"Professional degree (e.g. MD, DDS, DVM)",Laos,50.001 - 100.000 people,Other - Don't Know - Prefer Not To Answer - N/A,Home,Saturday,July,Logic Bombs,No,Yes,Laptop,Linux Operating System,Other - Don't Know - Prefer Not To Answer - N/A
2,Male,35-44 years old,Married or domestic partnership,Homemaker,€10.000 - €19.999,"Bachelor’s degree (e.g. BA, BS)",Australia,1.000.001 - 5.000.000 people,No,Office,Other - Don't Know - Prefer Not To Answer - N/A,September,Trojans,No,Yes,Desktop Computer,Other - Don't Know - Prefer Not To Answer - N/A,Other - Don't Know - Prefer Not To Answer - N/A
3,Other - Don't Know - Prefer Not To Answer - N/A,Other - Don't Know - Prefer Not To Answer - N/A,Other - Don't Know - Prefer Not To Answer - N/A,Employed full time (40 or more hours per week),€80.000 - €89.999,"Some college, no degree",Kyrgyzstan,101 - 1.000 people,Yes,Other - Don't Know - Prefer Not To Answer - N/A,Monday,January,Fileless malware,Other - Don't Know - Prefer Not To Answer - N/A,Other - Don't Know - Prefer Not To Answer - N/A,Laptop,Apple macOS,Other - Don't Know - Prefer Not To Answer - N/A
4,Other - Don't Know - Prefer Not To Answer - N/A,Under 12 years old,Married or domestic partnership,Self-employed,€50.000 - €59.999,"Bachelor’s degree (e.g. BA, BS)",Barbados,1.000.001 - 5.000.000 people,Yes,Office,Sunday,February,Other - Don't Know - Prefer Not To Answer - N/A,Yes,Yes,Tablet,Apple iOS,Other - Don't Know - Prefer Not To Answer - N/A


In [5]:
#choose the columns for the model
cat_col = ['Gender', 'Age Group', 'Marital Status', 'Employment Status',
       'Household Income', 'Level of Education',
       'Place of Residence Population',
       'Place of the last Incident', 'Day', 'Month', 'Type of Malware',
       'Economic Impacts', 'Technical Impacts', 'Type of Device',
       'Type of Operating System', 'Malware Protection']
dff=df[cat_col]
for cols in dff.columns:
    dff[cols]=dff[cols].astype('category')

C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
#define the X and Y
X = dff.copy()
y = df['Αttacked by a Cyber Attack']

In [7]:
#train test split to check validation accuracy
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
X_t, X_tt, y_t, y_tt = train_test_split(X, y, test_size=.25, random_state=150303,stratify=y,shuffle=True)


In [5]:
#lightgbm model
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(num_leaves= 63, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, num_class=3,
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)


lgbm.fit(X_t, y_t,eval_metric = 'logloss',eval_set = [(X_tt,y_tt)],categorical_feature=cat_col,early_stopping_rounds=100,verbose=100)


C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Age Group', 'Day', 'Economic Impacts', 'Employment Status', 'Gender', 'Household Income', 'Level of Education', 'Malware Protection', 'Marital Status', 'Month', 'Place of Residence Population', 'Place of the last Incident', 'Technical Impacts', 'Type of Device', 'Type of Malware', 'Type of Operating System']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\nib

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 1.09943
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 1.09862


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='None', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=4, num_class=3,
               num_leaves=63, objective=None, random_state=314, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=0.9,
               subsample_for_bin=200000, subsample_freq=0)

In [8]:
#catboost model
from catboost import CatBoostClassifier

catb = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    random_strength=43,
    depth=8,
    loss_function='MultiClass',
    eval_metric='Accuracy'
)

catb.fit(X_t, y_t,eval_set = [(X_tt,y_tt)],cat_features=cat_col,early_stopping_rounds=100,verbose=100)


0:	learn: 0.3334520	test: 0.3334480	best: 0.3334480 (0)	total: 2.4s	remaining: 19m 58s
100:	learn: 0.3362307	test: 0.3339040	best: 0.3352760 (3)	total: 5m 6s	remaining: 20m 9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.335276
bestIteration = 3

Shrink model to first 4 iterations.


In [7]:
#save lightgbm predictions
import os
k = os.getcwd()
#save lightgbm predictions
pred=lgbm.predict(X)
pred=pd.DataFrame(pred)
pred['True']=df['Αttacked by a Cyber Attack']
pred.columns=['y_pred','y_true']
k1 = k+'\\LightGBM.csv'
pred.to_csv(k1,index=False)

In [11]:
#save catboost predictions
import os
k = os.getcwd()
pred=catb.predict(X)
pred=pd.DataFrame(pred)
pred['True']=df['Αttacked by a Cyber Attack']
pred.columns=['y_pred','y_true']
k1 = k+'\\Catb.csv'
pred.to_csv(k1,index=False)

In [48]:
#save the light gbm model
from sklearn.externals import joblib 
k1 = k+'lightgbm.pkl'  
# Save the model as a pickle in a file 
joblib.dump(lgbm, k1) 


In [ ]:
#save the light gbm model
from sklearn.externals import joblib 
k1 = k+'catb.pkl'  
# Save the model as a pickle in a file 
joblib.dump(catb, k1) 
